In [26]:
import shap
import pandas as pd
import pickle
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from alibi.explainers import AnchorTabular
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import  train_test_split
from sklearn.feature_selection import SelectFromModel

In [27]:
import warnings
warnings.filterwarnings("ignore")

In [28]:
failures = pd.read_csv('../data/model_data/failures.csv',sep=',')
components = failures['component'].unique()

In [29]:
encoder = LabelEncoder()
for component in components:
    globals()[f"{component}_df"] = pd.read_csv(f'../data/model_data/labelled_data_{component}.csv',sep=',')
    globals()[f"{component}_df"]['turbine_id'] = encoder.fit_transform(['turbine_id']*globals()[f"{component}_df"].shape[0])
    # set the date as the index
    globals()[f"{component}_df"] = globals()[f"{component}_df"].set_index('timestamp')

In [30]:
class_target_name = "target_class"
for component in components:
    X = globals()[f"{component}_df"].drop(columns=['component',class_target_name])
    y = globals()[f"{component}_df"][class_target_name]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    globals()[f"{component}_X_train"] = X_train
    globals()[f"{component}_X_test"] = X_test
    globals()[f"{component}_y_train"] = y_train
    globals()[f"{component}_y_test"] = y_test

In [31]:
model_name = "xgb"

In [53]:
for component in components:
    with open ('../models/selected-{}.pickle'.format(component), 'rb') as file:
        globals()[f"{model_name}_{component}"] = pickle.load(file)


In [33]:
for component in components:
    globals()[f"{model_name}_{component}"].fit(globals()[f"{component}_X_train"], globals()[f"{component}_y_train"])
    globals()[f"{component}_params"] = globals()[f"{model_name}_{component}"].get_params()
    globals()[f"{component}_selector"] = SelectFromModel(globals()[f"{model_name}_{component}"], threshold=-np.inf, prefit=True, max_features=globals()[f"{component}_params"]['max_features'])
    globals()[f"{component}_selected_features"] = globals()[f"{component}_X_train"].columns[globals()[f"{component}_selector"].get_support()]
    globals()[f"{component}_selected_features_train"] = globals()[f"{component}_X_train"][globals()[f"{component}_selected_features"]]
    globals()[f"{component}_selected_features_test"] = globals()[f"{component}_X_test"][globals()[f"{component}_selected_features"]]
    globals()[f"{component}_selected_features_train"].reset_index(drop=True, inplace=True)
    globals()[f"{component}_selected_features_test"].reset_index(drop=True, inplace=True)
    

In [36]:
for component in components:
    globals()[f"{model_name}_{component}"].fit(globals()[f"{component}_selected_features_train"], globals()[f"{component}_y_train"])

In [37]:
for component in components:
    globals()[f"{component}_predit_fn"] = lambda x: globals()[f"{model_name}_{component}"].predict(globals()[f"{component}_selector"].transform(x))
    globals()[f"{component}_explainer"] = AnchorTabular(globals()[f"{component}_predit_fn"], globals()[f"{component}_selected_features_train"].columns)
    globals()[f"{component}_explainer"].fit(globals()[f"{component}_selected_features_train"].to_numpy())


In [38]:
GEARBOX_predit_fn = lambda x: xgb_GEARBOX.predict(x)
GEARBOX_explainer = AnchorTabular(GEARBOX_predit_fn, GEARBOX_selected_features_train.columns)
GEARBOX_explainer.fit(GEARBOX_selected_features_train.to_numpy())

GENERATOR_predit_fn = lambda x: xgb_GENERATOR.predict(x)
GENERATOR_explainer = AnchorTabular(GENERATOR_predit_fn, GENERATOR_selected_features_train.columns)
GENERATOR_explainer.fit(GENERATOR_selected_features_train.to_numpy())

GENERATOR_BEARING_predit_fn = lambda x: xgb_GENERATOR_BEARING.predict(x)
GENERATOR_BEARING_explainer = AnchorTabular(GENERATOR_BEARING_predit_fn, GENERATOR_BEARING_selected_features_train.columns)
GENERATOR_BEARING_explainer.fit(GENERATOR_BEARING_selected_features_train.to_numpy())

HYDRAULIC_predit_fn = lambda x: xgb_HYDRAULIC_GROUP.predict(x)
HYDRAULIC_explainer = AnchorTabular(HYDRAULIC_predit_fn, HYDRAULIC_GROUP_selected_features_train.columns)
HYDRAULIC_explainer.fit(HYDRAULIC_GROUP_selected_features_train.to_numpy())

TRANSFORMER_predit_fn = lambda x: xgb_TRANSFORMER.predict(x)
TRANSFORMER_explainer = AnchorTabular(TRANSFORMER_predit_fn, TRANSFORMER_selected_features_train.columns)
TRANSFORMER_explainer.fit(TRANSFORMER_selected_features_train.to_numpy())

AnchorTabular(meta={
  'name': 'AnchorTabular',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {'seed': None, 'disc_perc': (25, 50, 75)},
  'version': '0.9.7.dev0'}
)

In [39]:
def time_convert(sec):
  mins = sec // 60
  sec = sec % 60
  hours = mins // 60
  mins = mins % 60
  return "Time Lapsed = {0}:{1}:{2}".format(int(hours),int(mins),sec)

In [40]:
for component in components:
    globals()[f"{component}_test_np"] = globals()[f"{component}_selected_features_test"].to_numpy()
    globals()[f"{component}_test_length"] = globals()[f"{component}_test_np"].shape[0]
    globals()[f"anchors_{component}"] =[ ]
    


In [41]:
start_time = time.time()
for i, dp in enumerate(GEARBOX_test_np):
    anchors_GEARBOX.append(GEARBOX_explainer.explain(dp, threshold=0.95))
    current_time = time.time()
    time_lapsed = current_time - start_time
    print("{} von {} Schritten abgeschlossen. Zeit: {}".format(i, GEARBOX_test_length, time_convert(time_lapsed)), end="\r", flush=True)


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


In [42]:
for index, elem in enumerate(anchors_GEARBOX):
    print(index)
    print('Anchor: %s' % (' AND '.join(elem["data"]["anchor"])))
    print('Precision: %.2f' % elem["data"]["precision"])
    print('Coverage: %.2f' % elem["data"]["coverage"])

0
Anchor: 
Precision: 0.99
Coverage: 1.00
1
Anchor: grid_production_power_maximum > 2104.60
Precision: 1.00
Coverage: 0.24
2
Anchor: average_controller_hub_temperature <= 30.79
Precision: 1.00
Coverage: 0.50
3
Anchor: average_grid_rotor_inverter_phase_3_temperature > 38.69
Precision: 0.99
Coverage: 0.51
4
Anchor: average_controller_hub_temperature <= 26.50
Precision: 1.00
Coverage: 0.26
5
Anchor: average_ambient_temperature <= 16.43
Precision: 1.00
Coverage: 0.25
6
Anchor: average_controller_hub_temperature <= 26.50
Precision: 1.00
Coverage: 0.25
7
Anchor: average_absolute_ambient_wind_direction > 107.20
Precision: 0.99
Coverage: 0.75
8
Anchor: average_ambient_temperature <= 21.07
Precision: 1.00
Coverage: 0.50
9
Anchor: maximum_blades_pitch_angle > 89.60
Precision: 0.99
Coverage: 0.49
10
Anchor: maximum_blades_pitch_angle > 89.60
Precision: 0.99
Coverage: 0.49
11
Anchor: average_controller_hub_temperature <= 26.50
Precision: 1.00
Coverage: 0.25
12
Anchor: average_ambient_temperature <

In [43]:
start_time = time.time()
for i, dp in enumerate(GENERATOR_test_np):
    anchors_GENERATOR.append(GENERATOR_explainer.explain(dp, threshold=0.95))
    current_time = time.time()
    time_lapsed = current_time - start_time
    print("{} von {} Schritten abgeschlossen. Zeit: {}".format(i, GENERATOR_test_length, time_convert(time_lapsed)), end="\r", flush=True)

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


...

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


In [44]:
for index, elem in enumerate(anchors_GENERATOR):
    print(index)
    print('Anchor: %s' % (' AND '.join(elem["data"]["anchor"])))
    print('Precision: %.2f' % elem["data"]["precision"])
    print('Coverage: %.2f' % elem["data"]["coverage"])

0
Anchor: grid_production_voltage_phase_1_average > 399.24
Precision: 0.99
Coverage: 0.51
1
Anchor: hydraulic_group_oil_temperature_average <= 35.59
Precision: 0.96
Coverage: 0.50
2
Anchor: hydraulic_group_oil_temperature_average <= 30.30 AND average_spinner_temperature <= 27.48
Precision: 0.97
Coverage: 0.25
3
Anchor: average_nacelle_temperature <= 30.08 AND latest_average_reactive_power_generated_by_generator_0 > -451.61
Precision: 0.97
Coverage: 0.44
4
Anchor: grid_production_voltage_phase_1_average > 399.76
Precision: 0.99
Coverage: 0.24
5
Anchor: grid_production_voltage_phase_1_average > 398.80
Precision: 0.95
Coverage: 0.75
6
Anchor: grid_production_voltage_phase_1_average > 398.80
Precision: 0.95
Coverage: 0.75
7
Anchor: average_spinner_temperature > 27.48 AND latest_average_reactive_power_generated_by_generator_0 <= -244.49
Precision: 0.93
Coverage: 0.17
8
Anchor: grid_production_voltage_phase_1_average <= 398.80 AND grid_production_reactive_power_maximum > 1000.10 AND hydrauli

In [46]:
start_time = time.time()
for i, dp in enumerate(GENERATOR_BEARING_test_np):
    anchors_GENERATOR_BEARING.append(GENERATOR_BEARING_explainer.explain(dp, threshold=0.95))
    current_time = time.time()
    time_lapsed = current_time - start_time
    print("{} von {} Schritten abgeschlossen. Zeit: {}".format(i, GENERATOR_BEARING_test_length, time_convert(time_lapsed)), end="\r", flush=True)

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


...

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


In [47]:
for index, elem in enumerate(anchors_GENERATOR_BEARING):
    print(index)
    print('Anchor: %s' % (' AND '.join(elem["data"]["anchor"])))
    print('Precision: %.2f' % elem["data"]["precision"])
    print('Coverage: %.2f' % elem["data"]["coverage"])

0
Anchor: ambient_wind_speed_estimated_average <= 5.16
Precision: 0.97
Coverage: 0.51
1
Anchor: grid_production_voltage_phase_2_average <= 398.27
Precision: 0.99
Coverage: 0.75
2
Anchor: average_atmospheric_pressure > 1012.55
Precision: 1.00
Coverage: 0.25
3
Anchor: average_high_voltage_transformer_phase_1_temperature <= 64.12
Precision: 1.00
Coverage: 0.51
4
Anchor: average_high_voltage_transformer_phase_3_temperature <= 67.38
Precision: 1.00
Coverage: 0.51
5
Anchor: average_high_voltage_transformer_phase_1_temperature <= 64.12
Precision: 0.99
Coverage: 0.49
6
Anchor: average_high_voltage_transformer_phase_3_temperature <= 67.38
Precision: 1.00
Coverage: 0.50
7
Anchor: grid_production_voltage_phase_2_average <= 397.48
Precision: 1.00
Coverage: 0.51
8
Anchor: average_high_voltage_transformer_phase_3_temperature <= 67.38
Precision: 1.00
Coverage: 0.50
9
Anchor: grid_production_voltage_phase_2_average <= 396.94
Precision: 1.00
Coverage: 0.25
10
Anchor: grid_production_voltage_phase_3_ave

In [49]:
start_time = time.time()
for i, dp in enumerate(TRANSFORMER_test_np):
    anchors_TRANSFORMER.append(TRANSFORMER_explainer.explain(dp, threshold=0.95))
    current_time = time.time()
    time_lapsed = current_time - start_time
    print("{} von {} Schritten abgeschlossen. Zeit: {}".format(i, TRANSFORMER_test_length, time_convert(time_lapsed)), end="\r", flush=True)

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


...

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


In [50]:
for index, elem in enumerate(anchors_TRANSFORMER):
    print(index)
    print('Anchor: %s' % (' AND '.join(elem["data"]["anchor"])))
    print('Precision: %.2f' % elem["data"]["precision"])
    print('Coverage: %.2f' % elem["data"]["coverage"])

0
Anchor: grid_production_voltage_phase_2_average <= 398.27
Precision: 0.98
Coverage: 0.75
1
Anchor: grid_production_voltage_phase_2_average <= 398.27
Precision: 0.99
Coverage: 0.75
2
Anchor: grid_production_voltage_phase_2_average <= 397.48
Precision: 0.99
Coverage: 0.51
3
Anchor: hydraulic_group_oil_temperature_average <= 30.30
Precision: 1.00
Coverage: 0.25
4
Anchor: average_spinner_temperature <= 24.00
Precision: 1.00
Coverage: 0.50
5
Anchor: hydraulic_group_oil_temperature_average <= 30.30
Precision: 1.00
Coverage: 0.24
6
Anchor: hydraulic_group_oil_temperature_average <= 30.30
Precision: 1.00
Coverage: 0.25
7
Anchor: grid_production_voltage_phase_2_average <= 397.48
Precision: 1.00
Coverage: 0.50
8
Anchor: average_high_voltage_transformer_phase_3_temperature <= 67.38
Precision: 1.00
Coverage: 0.49
9
Anchor: hydraulic_group_oil_temperature_average <= 40.82
Precision: 1.00
Coverage: 0.75
10
Anchor: average_high_voltage_transformer_phase_3_temperature <= 77.19
Precision: 0.98
Covera

In [51]:
start_time = time.time()
for i, dp in enumerate(HYDRAULIC_GROUP_test_np):
    anchors_HYDRAULIC_GROUP.append(HYDRAULIC_explainer.explain(dp, threshold=0.95))
    current_time = time.time()
    time_lapsed = current_time - start_time
    print("{} von {} Schritten abgeschlossen. Zeit: {}".format(i, HYDRAULIC_GROUP_test_length, time_convert(time_lapsed)), end="\r", flush=True)

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


...

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


In [52]:
for index, elem in enumerate(anchors_HYDRAULIC_GROUP):
    print(index)
    print('Anchor: %s' % (' AND '.join(elem["data"]["anchor"])))
    print('Precision: %.2f' % elem["data"]["precision"])
    print('Coverage: %.2f' % elem["data"]["coverage"])

0
Anchor: grid_production_voltage_phase_2_average > 397.48 AND maximum_blades_pitch_angle <= 89.60
Precision: 0.96
Coverage: 0.24
1
Anchor: hydraulic_group_oil_temperature_average <= 35.59 AND average_grid_rotor_inverter_phase_3_temperature > 38.69
Precision: 0.97
Coverage: 0.12
2
Anchor: hydraulic_group_oil_temperature_average <= 30.30
Precision: 0.97
Coverage: 0.25
3
Anchor: hydraulic_group_oil_temperature_average <= 35.59 AND grid_production_voltage_phase_3_average > 397.14
Precision: 0.98
Coverage: 0.18
4
Anchor: grid_production_voltage_phase_2_average > 398.27
Precision: 0.99
Coverage: 0.25
5
Anchor: grid_production_voltage_phase_2_average > 398.27
Precision: 0.97
Coverage: 0.26
6
Anchor: hydraulic_group_oil_temperature_average <= 30.30
Precision: 0.95
Coverage: 0.26
7
Anchor: average_grid_rotor_inverter_phase_3_temperature <= 38.69 AND generator_bearing2_temperature_average > 41.39 AND generator_rotations_per_minute_std <= 65.87 AND grid_production_voltage_phase_2_average <= 397.